Link para melhor visualização do Notebook em seu ambiente de execução: [Notebook Colab](https://colab.research.google.com/drive/1pcckEs063FfUZgI9u61tKx7BNbwIEGMh?usp=sharing)


---


Notebook desenvolvido para construção de séries temporais baseada sem atualização de treinamentos. Os treinamentos incrementais e temporais precisam ter um modelo dado como "modelo de base", pois os seus treinamentos (séries) posteriores herdarão os parâmetros deste "treinamento base" para possibilitar algumas operações de comparação entre uma série e outra (principalmente similaridade de vetores que precisam ter a mesma quantidade de dimensões).
As séries temporais treinadas de forma temporal acontecem da seguinte forma:

1. Escolhe-se o Modelo base.
2. Ajusta-se os parâmetos para formação do corpus de atualização escolhendo as coleções (que deverão ser as mesmas do treinamento que originou o modelo base) e o intervalo de datas (o que, de fato, vai dizer quais textos das coleções selecionadas serão incrementados no treinamento do novo modelo).
3. Executa-se o treinamento temporal (sem atualização da base, apenas mudança no final do intervalo de datas na construção do corpus).
4. Aguarda-se **pacientemente** a finalização dos treinos.

*Observação: Este treinamento "temporal" tem como objetivo **remover a possibilidade de ocorrer o fenômeno de "esquecimento catastrófico"** que acomete atualizações de redes neurais nos treinamentos de inteligências artificiais. Pois, na lógica utilzada aqui não ocorre atualização, apenas adição de treinos com um corpus de textos maior a cada série temporal.*

*Um dos desenvolvedores do gensim-Word2Vec abordou temas relacionados à atualização de treinamentos para modelos Word2Vec e o próprio esquecimento catastrófico em uma de suas respostas em fóruns de dúvidas: [word2vec gensim update learning rate - StackOverflow - Gordon Mohr (aka "gojomo")](https://stackoverflow.com/questions/51133162/word2vec-gensim-update-learning-rate)*

# Preparação/Configuração de ambiente

In [ ]:
try:
    import os
    import re
    import shutil
    import msgpack

    from gensim.models import Word2Vec
    from gensim.models.callbacks import CallbackAny2Vec
    from gensim.utils import effective_n_jobs

    from google.colab import drive
    drive.mount('/content/drive')
    from google.colab import output
except Exception as e:
    erro = f'{e._class__.__name__}: {str(e)}'
    print(f'Erro ao configurar ambiente:\n--> {erro}')
else:
    print('Ambiente configurado com sucesso!')

Mounted at /content/drive
Ambiente configurado com sucesso!


# Funções

In [ ]:
def copiarArquivo(caminho_arquivo_original : str,
                  pasta_destino : str):
  """
  Função responsável por copiar um arquivo de um diretório para outro.

  ### Parâmetros:
  - caminho_arquivo_original: String contendo o caminho até o arquivo que se deseja
  copiar.
  - pasta_destino: String contendo o caminho até a pasta que receberá a cópia do
  arquivo.

  ### Retornos:
  - Bool que faz referência ao sucesso (True) ou não (False) da operação.
  """
  try:
    if not os.path.exists(pasta_destino):
      os.makedirs(pasta_destino)

    shutil.copy(caminho_arquivo_original, pasta_destino)
  except Exception as e:
    print(f'\n! Falha: {e.__class__.__name__}: {str(e)}')
    return False
  else:
    return True

def abrirArquivoMsgPack(full_filepath : str,
                        encoding_type : str = None):
    """
    Função responsável por abrir os arquivos no formato msgpack.

    ### Parâmetros:
    - full_filepath: String contendo o caminho completo até o arquivo que deseja-se
    abrir e extrair o conteúdo (variável salva).
    - encoding_type: String contendo o tipo de encoding, caso desejar.

    ### Retornos:
    - Variável salva (e agora aberta e lida) no arquivo msgpack.
    """
    if not full_filepath.endswith('.msgpack'):
        full_filepath += '.msgpack'
    if encoding_type:
        with open(full_filepath,'rb',encoding=encoding_type) as f:
            variable_bytes = f.read()
            variable_loaded = msgpack.unpackb(variable_bytes, raw=False)
            f.close()
            return variable_loaded
    else:
        with open(full_filepath,'rb') as f:
            variable_bytes = f.read()
            variable_loaded = msgpack.unpackb(variable_bytes, raw=False)
            f.close()
            return variable_loaded


class GeradorCorpusTokenizado:
    """
    Classe geradora de corpus amigável à memória RAM. Utiliza-se de iterador e gerador
    para não saturar a RAM do sistema que for executá-la. Desta forma não é necessário
    carregar todos os textos de todos os trabalhos e passar como parâmetros de frases
    na hora de treinar os modelos, pois a função de treino só precisa de um objeto
    "iterável" no parâmetro de frases (sentences). Sendo geradora só será carregado
    na RAM o texto que está se passando no momento ao invés do corpus de textos
    todos juntos ao mesmo tempo.

    ### Parâmetros:
    - intervalo_anos: Tupla de dois inteiros referentes ao ano de início e final
    da escrita dos textos que serão inseridos no corpus de alimentação de treino
    (ambos os extremos incluídos no intervalo).
    - colecoes: Lista de strings referentes às coleções que serão contempladas
    na criação do corpus de alimentação de treino.
    - caminho_pasta_colecoes_tokenizadas: Caminho até o corpus pré-processado onde
    será buscado as coleções, anos, trabalhos e arquivos de pré-processamento para
    alimentação do treinamento.
    - usando_reconhecimento_de_entidades: Bool que dirá se os arquivos de pré-processamento
    procurados serão os que usaram (True) ou não (False) o reconhecimento de entidades
    nos textos (atualmente o pré-processamento que foi totalmente concluído foi
    utilizando o reconhecimento de entidades).

    ### Retornos:
    - Objeto gerador e iterável sobre o corpus de textos contemplados pelos parâmetros
    ("intervalo_anos" e "colecoes") passados como entrada.
    """

    def __init__(self, intervalo_anos : tuple[int,int], usando_reconhecimento_de_entidades : bool = True, colecoes : list[str] | str ='todas', caminho_pasta_colecoes_tokenizadas : str =r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/Textos_pre_processados/Colecoes_textos_pre_processados'):
        self.intervalo_anos = range(intervalo_anos[0], intervalo_anos[1] + 1)
        self.usando_reconhecimento_de_entidades = usando_reconhecimento_de_entidades
        if usando_reconhecimento_de_entidades:
            self.arquivo_pre_processamento = 'pre_processamento_c_re.msgpack'
        else:
            self.arquivo_pre_processamento = 'pre_processamento_s_re.msgpack'
        if isinstance(colecoes, str):
            if colecoes.lower() == 'todas':
                colecoes = [c for c in os.listdir(caminho_pasta_colecoes_tokenizadas) if '.' not in c]
        self.arquivos = []
        for colecao in [os.path.join(caminho_pasta_colecoes_tokenizadas, c) for c in os.listdir(caminho_pasta_colecoes_tokenizadas) if c in colecoes]:
            lista_anos = sorted([a for a in os.listdir(colecao) if a.isdigit()])
            for ano in [os.path.join(colecao, a) for a in lista_anos if int(os.path.basename(a)) in self.intervalo_anos]:
                for trabalho in [os.path.join(ano, t) for t in os.listdir(ano) if t.startswith('Trabalho')]:
                    for arquivo in [os.path.join(trabalho, arq) for arq in os.listdir(trabalho) if arq == self.arquivo_pre_processamento]:
                        self.arquivos.append(arquivo)

    def __iter__(self):
        for cont,arquivo in enumerate(self.arquivos):
            for frase_tokenizada in abrirArquivoMsgPack(arquivo):
                yield frase_tokenizada


def extrairParametrosUsados(nome : str):
  """
  Função responsável por extrair os parâmetros utilizados no treinamento anterior
  para usar nos próximos.

  ### Parâmetros:
  - nome: String contendo o nome do arquivo de modelo treinado que possui os parâmetros
  utilizados.

  ### Retornos:
  - Tupla de dois elementos sendo o primeiro um bool de status de processo e o
  segundo um dicionário com as chaves sendo os parâmetros de treino e os valores
  sendo os valores dos parâmetros propriamente ditos (caso status do processo
  seja positivo, ou seja, True).
  """
  modo = re.search(r'modo\_(\d)',nome)
  if modo:
      modo = int(modo.group(1))
  else:
      modo = None

  dimensao = re.search(r'dimensao\_(\d+)',nome)
  if dimensao:
      dimensao = int(dimensao.group(1))
  else:
      dimensao = None

  negative = re.search(r'negative\_(\d+)',nome)
  if negative:
      negative = int(negative.group(1))
  else:
      negative = None

  window = re.search(r'window\_(\d+)',nome)
  if window:
      window = int(window.group(1))
  else:
      window = None

  epochs = re.search(r'epochs\_(\d+)',nome)
  if epochs:
      epochs = int(epochs.group(1))
  else:
      epochs = None

  alpha = re.search(r'alpha\_(\d\.\d+)',nome)
  if alpha:
      alpha = float(alpha.group(1))
  else:
      alpha = None

  min_count = re.search(r'min_count\_(\d+)',nome)
  if min_count:
      min_count = int(min_count.group(1))
  else:
      min_count = None
  if modo in [1,0] and dimensao and negative and window and epochs and alpha and min_count:
    resultado = {'modo':modo,
                 'dimensao':dimensao,
                 'negative':negative,
                 'window':window,
                 'epochs':epochs,
                 'alpha':alpha,
                 'min_count':min_count}

    return True, resultado
  else:
    return False, []

# Exec treino com intervalos de data estendidos

Começamos um treinamento com corpus de 2003 até 2006 (é o que já temos atualmente). Depois treinamos um modelo com o corpus de 2003 até 2008, depois de 2003 até 2010, 2003 até 2012, 2003 - 2014, até chegar em 2024 usando os mesmos parâmetros do treinamento "base" (de 2003 até 2006) para que possamos comparar os vetores de diferentes treinamentos entre si.

In [ ]:
# Escolhendo o corpus de treino e os modelos que melhor performaram nas analogias
# INFO MODELOS TODAS_2003_2006 (RI TODO COM INÍCIO EM 2003 - 2006)


# Modelo 1: modelo_modo_1_dimensao_500_negative_10_window_12_epochs_5_alpha_0.025_min_count_45.msgpack

# Modelo 2: modelo_modo_1_dimensao_300_negative_10_window_12_epochs_5_alpha_0.025_min_count_60.msgpack

# Modelo 3: modelo_modo_1_dimensao_300_negative_10_window_12_epochs_5_alpha_0.025_min_count_45.msgpack

# Modelo 4: modelo_modo_1_dimensao_500_negative_10_window_12_epochs_5_alpha_0.025_min_count_60.msgpack


# Criação de um dicionário que conterá a numeração e o nome do arquivo dos treinamentos
dic_melhores_modelos = {'1':'modelo_modo_1_dimensao_500_negative_10_window_12_epochs_5_alpha_0.025_min_count_45.msgpack',
                        '2':'modelo_modo_1_dimensao_300_negative_10_window_12_epochs_5_alpha_0.025_min_count_60.msgpack',
                        '3':'modelo_modo_1_dimensao_300_negative_10_window_12_epochs_5_alpha_0.025_min_count_45.msgpack',
                        '4':'modelo_modo_1_dimensao_500_negative_10_window_12_epochs_5_alpha_0.025_min_count_60.msgpack'}

# Nome central do modelo que terá as séries temporais
nome_central = 'UFSC'

# Nome completo do modelo que terá as séries temporais
nome_pasta_treino = 'Todas 2003 - 2006'

# Nome do modelo base das séries temporais
nome_modelo_treinado = 'UFSC_2003_2006'

# Caminho até a pasta que os modelos treinados foram salvos
caminho_pasta_modelos_salvos = os.path.join(r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/Treinamento do nosso modelo/Resultados Múltiplos Treinamentos',nome_pasta_treino)

# Caminho até a pasta que armazenará os modelos das séries temporais
caminho_pasta_treino_temporal = os.path.join(r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/Treinamento do nosso modelo/Treinamento com temporalização',nome_pasta_treino,'Com RE','Treinamento temporal')

# Criando a pasta de treino temporal
if not os.path.exists(caminho_pasta_treino_temporal):
  os.makedirs(caminho_pasta_treino_temporal)

# Armazenando num txt as informações dos modelos que melhor performaram que terão suas séries temporais construídas
caminho_txt_info = os.path.join(caminho_pasta_treino_temporal,'Arquivo de info dos Modelos.txt')
if not os.path.exists(caminho_txt_info):
  txt = 'INFO MODELOS\n\n'
  for n_modelo in dic_melhores_modelos.keys():
    txt += f"\nModelo {n_modelo}: {dic_melhores_modelos[n_modelo]}\n"
  with open(caminho_txt_info,'w',encoding='utf-8') as f:
    f.write(txt)

# Obtenção da quantidade de threads disponíveis no processador do sistema que está executando o programa
n_workers = effective_n_jobs(-1)

# Criação de classe para obtenção de informação durante os novos treinos
loss_list = []
class Callback(CallbackAny2Vec):
  def __init__(self):
      self.epoch = 0

  def on_epoch_end(self, model):
      loss = model.get_latest_training_loss()
      loss_list.append(loss)
      print('Loss after epoch {}:{}'.format(self.epoch, loss))
      model.running_training_loss = 0.0
      self.epoch = self.epoch + 1

# Processo para salvar o modelo base como modelo inicial nas séries temporais
# for n_modelo in dic_melhores_modelos.keys():
for n_modelo in list(dic_melhores_modelos.keys())[1:3]: # [1:3] (foi criado outros programas deste para rodarem outra parte da lista, pois treinamento temporal demora muito mais que o incremental)
  output.clear()
  print('Passando pelo modelo:',n_modelo)

  status_parametros, dic_parametros = extrairParametrosUsados(dic_melhores_modelos[n_modelo])

  if status_parametros:

    caminho_pasta_treino_temporal_n_modelo = os.path.join(caminho_pasta_treino_temporal,f'Modelo {n_modelo}')

    if not os.path.exists(caminho_pasta_treino_temporal_n_modelo):
      os.makedirs(caminho_pasta_treino_temporal_n_modelo)

    nome_arquivo_modelo_base = dic_melhores_modelos[n_modelo].replace('.msgpack','')

    caminho_save_modelo_base = os.path.join(caminho_pasta_modelos_salvos,nome_arquivo_modelo_base)

    caminho_modelo_base = os.path.join(caminho_pasta_treino_temporal_n_modelo,nome_arquivo_modelo_base)

    caminho_modelo_base_atualizado = os.path.join(caminho_pasta_treino_temporal_n_modelo,f'WOKE_{n_modelo}_{nome_modelo_treinado}_w2v_tmp.model')

    if not os.path.exists(os.path.join(caminho_pasta_treino_temporal_n_modelo,f'WOKE_{n_modelo}_{nome_modelo_treinado}_w2v_tmp.model')):
      for arquivo in [os.path.join(caminho_pasta_modelos_salvos,arq) for arq in os.listdir(caminho_pasta_modelos_salvos)]:
        if nome_arquivo_modelo_base in os.path.basename(arquivo):
          copiarArquivo(caminho_arquivo_original=arquivo,pasta_destino=caminho_pasta_treino_temporal_n_modelo)
          os.rename(os.path.join(caminho_pasta_treino_temporal_n_modelo,os.path.basename(arquivo)),os.path.join(caminho_pasta_treino_temporal_n_modelo,os.path.basename(arquivo).replace(nome_arquivo_modelo_base,f'WOKE_{n_modelo}_{nome_modelo_treinado}_w2v_tmp')))



    # Seleção das coleções que serão contempladas no corpus atualizado para o novo treinamento

    # SAUDE-CORPO
    # lista_de_colecoes = ['Biologia_Celular_e_do_Desenvolvimento','Biotecnologia_e_Biociencias','Ciencias_da_Reabilitacao','Ciencias_Medicas','Cuidados_Intensivos_e_Paliativos_Mestrado_Profissional',
    #                  'Educacao_Fisica','Enfermagem','Gestao_do_Cuidado_em_Enfermagem','Gestao_do_Cuidado_em_Enfermagem_Mestrado_Profissional','Medicina_Veterinaria_Convencional_e_Integrativa',
    #                  'Neurociencias','Saude_Coletiva','Saude_Mental_e_Atencao_Psicossocial_Mestrado_Profissional','Saude_Publica','Programa_de_Pos_Graduacao_Multidisciplinar_em_Saude_Mestrado_Profissional']

    # CFH
    # lista_de_colecoes = ['Filosofia','Geografia','Geologia','Historia','Psicologia','Teses_e_dissertacoes_do_Centro_de_Filosofia_e_Ciencias_Humanas','Programa_de_Pos_Graduacao_Interdisciplinar_em_Ciencias_Humanas','Servico_Social','Sociologia_e_Ciencia_Politica','Sociologia_Politica','Saude_Mental_e_Atencao_Psicossocial_Mestrado_Profissional','Ensino_de_Historia_Mestrado_Profissional']

    # UFSC
    lista_de_colecoes = 'todas'


    # Seleção dos intervalos das séries temporais que serão construídas tendo o modelo anterior como base (a primeira terá o modelo base como anterior)
    lista_intervalo_de_datas_posteriores = [(2003,2008),(2003,2010),(2003,2012),(2003,2014),(2003,2016),(2003,2018),(2003,2020),(2003,2022),(2003,2024)]


    # Construção das séries temporais depois do modelo selecionado como o primeiro
    for intervalo_de_datas in lista_intervalo_de_datas_posteriores:
      data_ini = intervalo_de_datas[0]
      data_fim = intervalo_de_datas[1]

      print('\nIntervalo de datas de treino atual:',data_ini,'-',data_fim)

      if not os.path.exists(os.path.join(caminho_pasta_treino_temporal_n_modelo,f'WOKE_{n_modelo}_{nome_central}_{str(data_ini)}_{str(data_fim)}_w2v_tmp.model')):

        corpus_atual = GeradorCorpusTokenizado(intervalo_anos=intervalo_de_datas, colecoes=lista_de_colecoes)

        modelo = Word2Vec(sentences=corpus_atual,sg=dic_parametros['modo'],vector_size=dic_parametros['dimensao'],
                          negative=dic_parametros['negative'],window=dic_parametros['window'],epochs=dic_parametros['epochs'],
                          alpha=dic_parametros['alpha'],min_count=dic_parametros['min_count'],workers=n_workers,
                          compute_loss=True,callbacks=[Callback()])

        modelo.save(os.path.join(caminho_pasta_treino_temporal_n_modelo,f'WOKE_{n_modelo}_{nome_central}_{str(data_ini)}_{str(data_fim)}_w2v_tmp.model'))
      else:
        print('Treinamento encontrado para:',f'WOKE_{n_modelo}_{nome_central}_{str(data_ini)}_{str(data_fim)}_w2v_tmp.model')

  else:
    print('Parâmetros não foram extraídos com êxito!')

# O loss dos treinos não deve ser um parâmetro tão importante como tratado por um dos desenvolvedores da gensim num fórum de dúvidas: https://stackoverflow.com/questions/73891182/why-does-the-loss-of-word2vec-model-trained-by-gensim-at-first-increase-for-a-fe
# Resumidamente, nas palavras dele (Gordon Mohr ou "gojomo"):
# "[...] Do the vectors work well on task-specific evaluations? That, moreso than loss trends, is the reliable indicator of whether your approach is working. [...]"


Passando pelo modelo: 3

Intervalo de datas de treino atual: 2003 - 2008
Treinamento encontrado para: WOKE_3_UFSC_2003_2008_w2v_tmp.model

Intervalo de datas de treino atual: 2003 - 2010
Treinamento encontrado para: WOKE_3_UFSC_2003_2010_w2v_tmp.model

Intervalo de datas de treino atual: 2003 - 2012
Treinamento encontrado para: WOKE_3_UFSC_2003_2012_w2v_tmp.model

Intervalo de datas de treino atual: 2003 - 2014
Treinamento encontrado para: WOKE_3_UFSC_2003_2014_w2v_tmp.model

Intervalo de datas de treino atual: 2003 - 2016
Treinamento encontrado para: WOKE_3_UFSC_2003_2016_w2v_tmp.model

Intervalo de datas de treino atual: 2003 - 2018
Treinamento encontrado para: WOKE_3_UFSC_2003_2018_w2v_tmp.model

Intervalo de datas de treino atual: 2003 - 2020
Treinamento encontrado para: WOKE_3_UFSC_2003_2020_w2v_tmp.model

Intervalo de datas de treino atual: 2003 - 2022
Loss after epoch 0:75319720.0
Loss after epoch 1:74963976.0
Loss after epoch 2:74799632.0
Loss after epoch 3:74270568.0
Loss afte